In [61]:
import pandas as pd

#seed for consistent results across runtime
# seed_int = 3
# random.seed(seed_int)

#This code is for combining certain data from the necessary csv files into a single dataframe (complete)

#Data source:
#https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset?select=movies_metadata.csv

pd.set_option('display.max_colwidth', None)

movies_full = pd.read_csv('newdata/movies_metadata.csv',usecols=("genres","id" ,"title","tagline", "overview","production_companies"),
                          dtype={"tagline": "string", "id":"string", 'genres':"string", "title": "string", "tagline": "string","overview":"string", "production_companies" :"string"})
ratings = pd.read_csv('newdata/ratings.csv', usecols = ("userId", "movieId", "rating"), dtype={"userId": "string","movieId": "string","rating": "string"})
ratings = ratings.rename(columns={"movieId": "id"})

keywords = pd.read_csv('newdata/keywords.csv', usecols = ("id", "keywords"), dtype={"id": "string","keywords":"string"})
credits = pd.read_csv("newdata/credits.csv", usecols = ("cast", "id"), dtype={"cast": "string", "id": "string"})

complete =  pd.merge(movies_full, ratings, on ="id")
complete =  pd.merge(complete,keywords, on ="id")
complete  = pd.merge(complete,credits, on ="id")


#new:
#userIds are in order
# every_id = list(unique(list(complete["userId"])))
#userIds are out of order
# sample_ids  = random.sample(every_id, 1000)
# completeNew = pd.DataFrame()
#This is a very expensive task...
#it is possile to choose a subset of users from here instead of
#ibcluing  the entire set of users
# for user in sample_ids:
#     completeNew = pd.concat([completeNew, complete.loc[complete["userId"] == user]])

#new:
# complete = complete.sample(frac=1, random_state = seed_int, axis =0)
#this is not the desired behavior
#the users ids need to show up in a true random order
# complete = complete.groupby(by = "userId", sort = False, group_keys = True).apply(lambda x: x)
#this is omitted since the values should not be sorted by userId just grouped by userId

complete = complete.sort_values(by = 'userId')

complete  = complete.dropna()

complete  = complete.loc[:,['userId','id','rating',"title", "genres","production_companies","keywords", "cast", "tagline", "overview" ]]


print(complete.head())

# f = open("test_dicts.txt", "w", encoding="utf-8")
# f.write(str(list(complete["tagline"])))
# f.write(str(list(complete["overview"])))
# f.close()


        userId    id rating               title  \
8830418      1  2959    4.0      License to Wed   
1690638      1  1968    4.0       Fools Rush In   
2343877      1  2762    4.5  Young and Innocent   
2943547      1   147    4.5       The 400 Blows   
8434697      1  1246    5.0        Rocky Balboa   

                                                                                                genres  \
8830418                                                                 [{'id': 35, 'name': 'Comedy'}]   
1690638  [{'id': 18, 'name': 'Drama'}, {'id': 35, 'name': 'Comedy'}, {'id': 10749, 'name': 'Romance'}]   
2343877                                     [{'id': 18, 'name': 'Drama'}, {'id': 80, 'name': 'Crime'}]   
2943547                                                                  [{'id': 18, 'name': 'Drama'}]   
8434697                                                                  [{'id': 18, 'name': 'Drama'}]   

                                                        

In [84]:
import ast
import random

#seed for consistent results across runtime
# seed_int = 3
# random.seed(seed_int)

#used to filter out the rows of data with empty entries
def condition(array):
    length = len(array[4])
    if(array[4][length-2:] == "[]"):
        return False
    length = len(array[5])
    if(array[5][length-2:] == "[]"):
        return False
    length = len(array[6])
    if(array[6][length-2:] == "[]"):
        return False
    length = len(array[7])
    if(array[7][length-2:] == "[]"):
        return False   
    #this is probably not needed due to the dropNa function used above...
    # length = len(array[8])
    # if(array[8][length-4:]=="<NA>"):
    #     return False
    # length = len(array[9])
    # if(array[9][length-4:]=="<NA>"):
    #     return False 
    return True


#used to extract names
def populate_names(item):
    string  = item[1:-1]
    jsons = string.split("}, ")   
    names = ""
    cnt = 0
    for item in jsons:
        if(cnt == len(jsons)-1):
            temp_dict = ast.literal_eval(item)
            names+=str(temp_dict["name"])
        else:
            temp_dict = ast.literal_eval(item+"}")
            names+=str(str(temp_dict["name"])+" ")
        cnt += 1
    return names

#extract data from row of complete_array
def provide_data(array):
    movie_data = []
    movie_data.append(int(array[0]))
    movie_data.append(int(array[1]))
    movie_data.append(float(array[2]))
    movie_data.append(array[3])  

    movie_data.append(populate_names(array[4]))
    movie_data.append(populate_names(array[5]))
    movie_data.append(populate_names(array[6]))
    movie_data.append(populate_names(array[7]))

    movie_data.append(str(array[8]))
    movie_data.append(str(array[9]))
    return movie_data
    


#new method
list_of_user_ids = list(complete["userId"].unique())
# print(list_of_user_ids)

counts = complete['userId'].value_counts()
# print(counts)

gaps = [counts[id] for id in list_of_user_ids]
# print(gaps)

# gaps = []
# for id in list_of_user_ids:
#     gaps.append(counts[id])

complete_array = complete.to_numpy()

# print(complete)
# print(complete_array)



#old method...
# gaps = []
# size = 0
# list_of_user_ids = []
# last_id  = -1
# past_first_it = False


# for row in complete_array:
#     if(row[0]!= last_id):
#         list_of_user_ids.append(row[0])
#         last_id = row[0]
#         if(past_first_it ==True):
#             gaps.append(size)
#             size =0 
#     size+=1
#     past_first_it = True

# #there is always a gap for the last iteration
# gaps.append(size)



index  = 0

user_to_data = []
#this is the total number of users in the whole dataset
#total number of users: 261306
total_nof_users = len(list_of_user_ids)
#this is the number of desired users before filtering
#note: set back to 40000
desired_nof_users_before_filter = 61306

print(total_nof_users)

avg = 0.0
cnt = 0.0


#instead of using this random pass generaion 
#the users can be picked randomly before hand and replace list_of_user_ids[i]
#will index still work???

#populate user_to_data from complete_array
for i in range(0, total_nof_users):
    #generate a random float to determine a pass for the user
    if (random.random()<float(desired_nof_users_before_filter/total_nof_users)):
        # if(gaps[i] >= 50 and gaps[i]<=75):
        if(gaps[i] >= 100):
            user_to_data.append([])
            last_index = len(user_to_data) -1
            for j in range(index, len(complete_array)):
                if complete_array[j][0] == list_of_user_ids[i]:
                    #condition is checked for complete_array[j]
                    if(condition(complete_array[j])):
                        #this is where data is tranformed
                        transformed = provide_data(complete_array[j])
                        #is copy needed here???
                        user_to_data[last_index].append(transformed)

                    #last iteration
                    # if (j==len(complete_array)-1):
                        #note: what if a similair condition could be checked before running this loop to prevent redundant processing
                        # if (len(user_to_data[last_index])<50 or len(user_to_data[last_index])>75):
                        # if (len(user_to_data[last_index])<100):
                        #     del user_to_data[last_index]       
                else:
                    avg += len(user_to_data[last_index])
                    cnt+=1
                    #note: what if a similair condition could be checked before running this loop to prevent redundant processing
                    #if (len(user_to_data[last_index])<50 or len(user_to_data[last_index])>75):
                    # if (len(user_to_data[last_index])<100):
                    #     del user_to_data[last_index]  
                    #note: changed from (index = j+1)
                    index = j
                    break
        else:
            index += gaps[i]             
    else:
        index += gaps[i]


#last iteration...



#runtime test, go through user_to_data and re-index the users in list order
for i in range(len(user_to_data)):
    for j in range(len(user_to_data[i])):
        user_to_data[i][j][0] = i



#needs to be sure that there are enough users after the condiiton
print(len(user_to_data))

#average number of ratings per users
print(float(avg/cnt))


#gaps[i] >= 100
#desired_nof_users_before_filter = 61306
#time taken: 11 minutes
#average rating: 176.65865842055186
#number of users: 4204


261306
4204
176.65865842055186


In [85]:
#save in a file so that cells below can run without running this cell and above

#question: would renaming the user ids as indexes in their order be helpful???

import csv

with open("constructedData/constructedData.csv", "w", encoding="utf-8", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['userId','id','rating',"title", "genres","production_companies","keywords", "cast", "tagline", "overview"])
    for i in range(len(user_to_data)):
        writer.writerows(user_to_data[i])

In [10]:
#this is a starting point if the data is already saved to the constructedData.csv file
import csv

data_list =[]

with open("constructedData/constructedData.csv", 'r', encoding="utf-8") as read_obj:
    csv_reader = csv.reader(read_obj)
    data_list = list(csv_reader)

data_list = data_list[1:]


In [11]:
import random

#try removing unecessay copying...

#seed for consistent results across runtime

seed_int = 2
random.seed(seed_int)

#user to data rows 
user_to_data = []
user_to_data_train = []
user_to_data_test = []
user_id = -1

#note: works when row[0] is also an index
for row in data_list:
    if (row[0]!=user_id):
        user_id = row[0]
        user_to_data.append([row])
    else:
        user_to_data[int(row[0])].append(row)


#this can be tweaked...
for i in range(3600):
    index = random.randint(0, len(user_to_data)-1)
    user_to_data_train.append(user_to_data[index])
    del user_to_data[index]


#for test data to be used later...
for i in range(600):
    index = random.randint(0, len(user_to_data_train)-1)
    user_to_data_test.append(user_to_data_train[index])
    del user_to_data_train[index]


del user_to_data


In [21]:
from gensim.parsing.preprocessing import remove_stopwords
from nltk.stem import WordNetLemmatizer
import random
import json
from ordered_set import OrderedSet
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cluster import KMeans

#the version from numpy is used instead
from scipy import linalg
from scipy.linalg import sqrtm
import math
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import linear_kernel



#seed for consistent results across runtime
# seed_int = 1
# random.seed(seed_int)


#check if there are variables that can be removed!!!
class user_type_vars():
    def __init__(self):
        #user to movie_id to a list of all the words in that movie (copies allowed)
        self.user_to_movie_id_to_corpus = [] 

        #user to movie_id to its rating
        self.user_to_movie_id_to_rating = [] 

        #for each user, includes a random choice from all the movies the user watched to be the target
        self.user_to_target_movie_id = [] 

        #for each user, the index is the placement of the target movie in the entire set of train movies inorder
        self.user_to_target_index_full = [] 

        #for each user, a set of all the words in all the movies the user watched 
        self.user_to_words_in_order  = [] 

        #for each user, the word counts for movies that they watched
        self.user_to_word_counts  = [] 
        self.user_to_word_counts_transformed = []

        #The word counts of the target movie for each user
        self.user_to_target_word_counts  = [] 
        self.user_to_target_word_counts_transformed = []

        #for each user, includes all the ratings of movies the user has rated with a -1 marked as the target rating
        self.user_to_ratings  = []

        #for each user, includes ratings for all the movies in the entire train set (missing ratings are filled in)
        self.user_to_ratings_full = [] 
        self.user_to_ratings_full_transform = []

        #model features x
        self.feature_1 = []
        self.feature_2 = []
        self.feature_3 = []

        #output feature y
        self.user_to_target_rating  = [] 


#need to think about the order these variables are used to improve memory perf...
#goal: minimize the window between when a variable was populated and used


#user_to_movie_id_to_corpus:
#1,2, 3, 4

#user_to_movie_id_to_rating:
#1, 3, after 5, 6

#user_to_target_movie_id:
#1, 3, 4, after 5, 6

#user_to_target_index_full:
#6,7

#user_to_words_in_order:
#2,4
#this seems optimal

#user_to_word_counts:
#4,5,

#user_to_word_counts_transformed:
#4,5,

#user_to_target_word_counts:
#4,5

#user_to_target_word_counts_transformed:
#4,5

#user_to_ratings: (the combination of 4 and 5 streamline this this)
#3,5

#user_to_ratings_full:
#6,7

#user_to_ratings_full_transform:
#6,7

#needed until the end:
#self.features...
#self.user_to_target_rating


#goal: minimize the window between when a vraible was populated and used
#or remove intermediate variables 


# first observation:
# these variables are populated and used in sucession
# user_to_word_counts:
# user_to_word_counts_transformed:
# user_to_target_word_counts:
# user_to_target_word_counts_transformed:

# the sole purpose of 4 and 5 is to created a cossine similairty list
# so populate 4 and 5 can be combined


#idea:
#exchange places of 2 and 3


#idea:
#after the switch is populate 3 even necessary ???


        
train_users = user_type_vars()
test_users = user_type_vars()


#no changes needed for these vars below...
movie_id_to_ratings_train = dict() #all ratings for each movie
movie_id_to_ratings_test = dict()
# movies_in_order = OrderedSet() #all movies in order

train_movies_in_order = OrderedSet()
test_movies_in_order = OrderedSet()

user_to_average_rating = []



wnl = WordNetLemmatizer()

#train_users.user_to_movie_id_to_rating
#train_users.user_to_target_movie_id
#test_users.user_to_movie_id_to_rating
#test_users.user_to_target_movie_id


def load_feature_1(movies_in_order, user_to_data, movie_id_to_ratings, user_to_movie_id_to_rating, user_to_target_movie_id, user_to_target_rating, feature_1, feature_2):
    #how do I integrate pop the other populate functions???
    overall_rating_sum = 0
    overall_rating_count = 0
    for i in range(len(user_to_data)):
        movie_id_to_words_temp = dict()
        movie_id_to_rating_temp = dict()
        cnt = 0
        total =0
        rand_int = random.randint(0, len(user_to_data[i])-1)
        for movie_data in user_to_data[i]:
            if cnt == rand_int:    
                user_to_target_movie_id.append(movie_data[1])
            else:
                overall_rating_sum += float(movie_data[2])
                overall_rating_count += 1
                total += float(movie_data[2])
            
            if movie_data[1] in movie_id_to_ratings.keys():
                movie_id_to_ratings[movie_data[1]].append(float(movie_data[2]))
            else:
                movie_id_to_ratings[movie_data[1]] = [float(movie_data[2])]

            movie_string = ""
            #avoid the first three data points (user id, movieid, and rating)
            #use only the text data...

            #use all the test data...
            # for index in range (3,len(movie_data)):
            #     if(index!= len(movie_data)-1):
            #         movie_string+= movie_data[index]+" "
            #     else:
            #         movie_string+= movie_data[index]

            #only use the genres...
            #all of the corpus and a few single columns were tested and with multple...
            #technique the simple complete and unweighted average proved as a better featur einthe model than
            movie_string = movie_data[4]

            #lematization
            cleaned = remove_stopwords(movie_string)
            cleaned = [wnl.lemmatize(word) for word in cleaned.split(" ")]
            cleaned = [word[:-1] for word in cleaned if word.endswith(".")] + [word for word in cleaned if not word.endswith(".")]
            #is copy really needed with this scope ???
            movie_id_to_words_temp[movie_data[1]] = cleaned
            movie_id_to_rating_temp[movie_data[1]] = float(movie_data[2])
            movies_in_order.add(movie_data[1])
            cnt+=1
        user_to_movie_id_to_rating.append(movie_id_to_rating_temp)
        user_to_average_rating.append(float(total/(cnt-1)))

        #the current users list of words from all the movies they rated
        users_words_in_order = OrderedSet()
        for movie_id in movie_id_to_words_temp.keys():
            for word in movie_id_to_words_temp[movie_id]:
                users_words_in_order.add(word)


        word_counts = []
        target_word_counts = []

        # these are only relevant with user averages scalingas opposed to movie average scaling...
        word_counts_transformed = []
        target_word_counts_transformed = []

        #this will transform into the word count averages for each word for each movie rated by the user
        averages = dict()

        #for each movie the user watched record the wordcount for each word in the movies the user watched
        for movie_id in movie_id_to_words_temp.keys():
            if movie_id != user_to_target_movie_id[-1]:
                temp_dict = Counter(movie_id_to_words_temp[movie_id])
                temp_list = []
                # avg = 0
                # cnt =0
                for word in users_words_in_order:
                    if word in temp_dict.keys():
                        temp_list.append(temp_dict[word])
                        # avg+=temp_dict[word]
                        # cnt+=1
                        if word in averages.keys():
                            averages[word] += temp_dict[word] 
                        else:
                            averages[word] = temp_dict[word] 
                    else:
                        temp_list.append(0) 
                        if word in averages.keys():
                            averages[word] += 0 
                        else:
                            averages[word] = 0                        
                #option 1:
                # avg = float(avg/len(users_words_in_order))
                #option 2:
                # avg = float(avg/cnt)
                word_counts.append(temp_list)
                #option 1:
                # temp_list_normalized = []
                # for item in temp_list:
                #     if item == 0:
                #         temp_list_normalized.append(0)
                #     else:
                #         temp_list_normalized.append(item - avg)
                # word_counts_transformed.append(temp_list_normalized)
                #option 2:
                # word_counts_transformed.append([x - avg for x in temp_list])
            else:

                temp_dict = Counter(movie_id_to_words_temp[movie_id])
                temp_list = []
                # avg = 0
                # cnt =0
                for word in users_words_in_order:
                    if word in temp_dict.keys():
                        temp_list.append(temp_dict[word])
                        # avg+=temp_dict[word]
                        # cnt +=1
                        if word in averages.keys():
                            averages[word] += temp_dict[word] 
                        else:
                            averages[word] = temp_dict[word]             
                    else:
                        temp_list.append(0)

                        if word in averages.keys():
                            averages[word] += 0 
                        else:
                            averages[word] = 0
                # option 1:
                # avg = float(avg/len(users_words_in_order))
                # option 2:
                # avg = float(avg/cnt)
                target_word_counts = temp_list
                #option 1:
                # target_word_counts_transformed = []
                # for item in temp_list:
                #     if item == 0:
                #         target_word_counts_transformed.append(0)
                #     else:
                #         target_word_counts_transformed.append(item - avg)
                #option 2:
                #target_word_counts_transformed = [x - avg for x in temp_list]
        

        #create actual averages list... 
        #lambda can be used here...
        def scale(x):
            return float(x/len(movie_id_to_words_temp))
        averages = [scale(averages[key]) for key in averages.keys()]

        #option 1:
        # complete = word_counts.copy()
        # complete.append(target_word_counts)
        # movie_averages = np.tile(averages, (len(complete),1))
        # complete_array = np.array(complete)
        # transformed_list  = list(complete_array - movie_averages)

        #option 2:
        complete_word_counts = word_counts.copy()
        complete_word_counts.append(target_word_counts)
        # tfidf = TfidfTransformer().fit_transform(complete_word_counts).toarray()
        tfidf = complete_word_counts

        #for content based reccomendation, may need to focus on words from certain sections to not overwelm the model with useless information!!!

        def svd_u(a, n):
            U, s, V = np.linalg.svd(a, full_matrices=False)
            #adjust with n
            s=np.diag(s)
            s=s[0:n,0:n]
            U=U[:,0:n]
            V=V[0:n,:]
            #could try returning UsV instead...
            #could try truncated svd...
            return list(U)
        
        #option 1:
        #transformed_word_counts = svd_u(transformed_list, 5)
        #option 2:
        #transformed_word_counts = svd_u(tfidf, 5)
        #option 3:
        #transformed_word_counts = TruncatedSVD(n_components = 20, random_state = seed_int).fit_transform(tfidf)


        #populate ratings with the exception of the target rating and also record
        #the users target rating
        ratings = []
        for movie_id in movie_id_to_rating_temp.keys():
            if movie_id != user_to_target_movie_id[-1]:
                ratings.append(movie_id_to_rating_temp[movie_id])
            else:
                #this signifies the ratings to be predicted by the model
                user_to_target_rating.append(movie_id_to_rating_temp[movie_id])
        
        #return the average ratings from movies that are alike the target movie...
        def predict(word_counts_transformed, word_counts,
            target_word_counts_transformed, target_word_counts, ratings):
            item_1 = 0 
            item_2 = 0
            item_3 = 0

            #option 0:
            # mdl = KMeans(n_clusters=5, n_init = "auto", random_state= seed_int)
            # predictions = mdl.fit_predict(list(transformed_word_counts))
            # target_cluster = predictions[len(predictions)-1]

            # same_cluster = list(filter((lambda item: item == target_cluster), predictions))

            # if(len(same_cluster)==1):
            #     #there are no other ratings in the target movies cluster...
            #     distance = np.inf
            #     index =0
            #     i = 0 
            #     for cord in mdl.cluster_centers_[0:-1]:
            #         if math.dist(cord, mdl.cluster_centers_[-1]) < distance:
            #             distance = math.dist(cord, mdl.cluster_centers_[-1])
            #             index = i
            #         i+=1
            #     closest_cluster = predictions[index]
            #     i =0
            #     cnt =0
            #     avg = 0
            #     for item in predictions[0:-1]:
            #         if item == closest_cluster:
            #             avg+= ratings[i]
            #             cnt+=1
            #         i+=1
            #     item_3 = float(avg/cnt)      

            # else:
            #     #there is at least one other movie in the target movies cluster
            #     i =0
            #     cnt =0
            #     avg = 0
            #     for item in predictions[0:-1]:
            #         if item == target_cluster:
            #             avg+= ratings[i]
            #             cnt+=1
            #         i+=1
            #     item_3 = float(avg/cnt)

            #option 1: 
            # cosine_sim = linear_kernel(X = tfidf[0:-1],Y = [tfidf[-1]])
            # cosine_sim = np.reshape(cosine_sim,  (len(cosine_sim)))
            # combined = zip(cosine_sim, ratings)
            # combined = sorted(combined, key=lambda x: x[0], reverse=True)
            # avg = 0
            # nof = 10.0
            # for i in range(int(nof)):
            #     avg += combined[i][1]
            # item_3 =  float(avg/nof)

            #option 2:
            avg = 0
            for i in range(len(ratings)):
                avg += ratings[i]
            item_1 = float(avg/len(ratings))

            #option 3:
            # cosine_sim = linear_kernel(X = transformed_list[0:-1],Y = [transformed_list[-1]])
            # cosine_sim = np.reshape(cosine_sim,  (len(cosine_sim)))
            # combined = zip(cosine_sim, ratings)
            # combined = sorted(combined, key=lambda x: x[0], reverse=True)
            # avg = 0
            # nof = 10.0
            # for i in range(int(nof)):
            #     avg += combined[i][1]
            # item_1 = float(avg/nof)

            #option 4:
            # cosine_sim = linear_kernel(X = tfidf[0:-1],Y = [tfidf[-1]])
            cosine_sim = cosine_similarity(X = tfidf[0:-1],Y = [tfidf[-1]])
            cosine_sim = np.reshape(cosine_sim,  (len(cosine_sim)))
            numerator = 0
            denominator = 0
            item_2 = item_1
            for i in range(len(ratings)):
                numerator += float(cosine_sim[i]*ratings[i])
                denominator += cosine_sim[i]
            
            if denominator != 0:
                item_2 = float(numerator/denominator)
        
            return (item_1, item_2)
        
        
        items = predict(word_counts_transformed, word_counts,
                                    target_word_counts_transformed, target_word_counts, ratings)

        # feature_1.append(items[0])
        feature_1.append(items[0])
        feature_2.append(items[1])
            
        
    return float(overall_rating_sum/overall_rating_count)





def pre_svd(movie_id_to_average_rating, movies_in_order, user_to_ratings_full_transform, user_to_ratings_full, user_to_target_index_full, 
               user_to_movie_id_to_rating, user_to_target_movie_id):
    for i in range(len(user_to_movie_id_to_rating)):
        ratings = []
        transformed_ratings = []
        index = 0


        #what if there is no movie_id == user_to_target_movie_id[i]
        #this can happen when a test users target movie is not in the train_movies_in_order...

        #solution:

        #this could be run once with only train_movies
        #and then used to populate the train svd
        #and then extract the prediction to train the model


        #then again with all movies train_movies + test_movies
        #then used to populate the full svd
        #and then extract the prediction to test model


        #note: movie_id_to_average_rating_train shouold onyl be used for the train run of this function
        #for the test version of the this movie_id_to_average_rating_full should be used

        for movie_id in movies_in_order:
            if movie_id == user_to_target_movie_id[i]:
                user_to_target_index_full.append(index)
                ratings.append(movie_id_to_average_rating[movie_id])
                #note: per item averages are being subtracted here instead of per user averages
                transformed_ratings.append(movie_id_to_average_rating[movie_id] - movie_id_to_average_rating[movie_id])             
            elif movie_id in user_to_movie_id_to_rating[i].keys():
                ratings.append(user_to_movie_id_to_rating[i][movie_id])
                #note: per item averages are being subtracted here instead of per user averages
                transformed_ratings.append(user_to_movie_id_to_rating[i][movie_id] - movie_id_to_average_rating[movie_id])
            else:
                ratings.append(movie_id_to_average_rating[movie_id])
                #note: per item averages are being subtracted here instead of per user averages
                transformed_ratings.append(movie_id_to_average_rating[movie_id] - movie_id_to_average_rating[movie_id])
            index +=1
        user_to_ratings_full.append(ratings)
        user_to_ratings_full_transform.append(transformed_ratings)


#note: before passing to this function the data is normalized about the average movie ratings (not average user ratings)
#each user train and test users have a single rating that needs to be trained against in the train case
#and predicted in the test case

#the svd can be applied to the combined data of the train and test sets
#both movies that the user didn't watch and movies that should be guesses are...
#transformed to have a value of zero before svd

#the movie columns are taken from the train dataset...
#senario: suppose a test user has a rating of a movie not part of the train set and it is not the target movie (ignore it)
#senario: suppose a test user has a rating of a movie not part of the train set and it is the target movie (guess the rating instead of using svd)

#...Once the UsV is created...
#take the rating from the new UsV for the user row and movie column for the target movie
#other option: cossine similairty on the U ignoring other test users


def svd_full(user_to_ratings_full_transform, n, movie_id_to_average_rating):
    U, s, V = np.linalg.svd(user_to_ratings_full_transform, full_matrices=False)
    
    #simplify ratings to n features
    s=np.diag(s)
    s=s[0:n,0:n]
    U=U[:,0:n]
    V=V[0:n,:]

    #reconstrcut to a new array
    Us = np.dot(U,s)
    UsV = np.dot(Us,V)
    

    #the keys of movie_id_to_ratings is in the same order of movies_in_order and therefore so is movie_id_to_average_rating_train
    x = np.tile(list(movie_id_to_average_rating.values()), (UsV.shape[0],1))

    #this tranforms the UsV row by row into the original rating scale (1-5)
    UsV = UsV + x

    #be consistent with data structures...
    return list(UsV)



overall_average_train = load_feature_1(train_movies_in_order, user_to_data_train, movie_id_to_ratings_train, train_users.user_to_movie_id_to_rating, 
                                                         train_users.user_to_target_movie_id, train_users.user_to_target_rating, train_users.feature_1, train_users.feature_2)


load_feature_1(test_movies_in_order, user_to_data_test, movie_id_to_ratings_test, test_users.user_to_movie_id_to_rating, 
               test_users.user_to_target_movie_id,
               test_users.user_to_target_rating, test_users.feature_1, test_users.feature_2)




#this has been replaced with load_feature_3...
#it has much lower precision than load_feature_3
#the feature simply provides an average rating from all users who rated the movie...
#or fills in the next best guess...
#note: with the current stack this depends on variables populated below it

# def load_feature_2(train_user, feature_2):
#     if(train_user):
#         #populates feature_2_train
#         for i in range(len(train_users.user_to_movie_id_to_rating)): 
#             if(len(movie_id_to_ratings_total[train_users.user_to_target_movie_id[i]])==1):
#                 #this means that there is no other rating besides the first train rating
#                 feature_2.append(overall_average_train)
#             else:
#                 #omit the rating the user in question made
#                 feature_2.append(float(((movie_id_to_average_rating_train[train_users.user_to_target_movie_id[i]]
#                                 *len(movie_id_to_ratings_total[train_users.user_to_target_movie_id[i]]))
#                                 -train_users.user_to_movie_id_to_rating[i][train_users.user_to_target_movie_id[i]])
#                                 /(len(movie_id_to_ratings_total[train_users.user_to_target_movie_id[i]])-1)))
#     else:   
#         #populates feature_2_test
#         for i in range(len(test_users.user_to_movie_id_to_rating)):
#             if(test_users.user_to_target_movie_id[i] not in movie_id_to_ratings_total.keys()):
#                 #if the movie is not in the train set make a guess
#                 feature_2.append(overall_average_train)
#             else:
#                 #only use the train data
#                 feature_2.append(movie_id_to_average_rating_train[test_users.user_to_target_movie_id[i]])


# depreciated...
# load_feature_2(True, train_users.feature_2)
# load_feature_2(False, test_users.feature_2)


#Unlike the other feature loading functions it only makes sense to run this once since...
#there is significantly difference processes for train and test data
def load_feature_3():

    movie_id_to_average_rating_train = dict()
    movie_id_to_average_rating_full = dict()

    #is all_movies_in_order still in order???
    all_movies_in_order = train_movies_in_order|test_movies_in_order

    for movie in all_movies_in_order:
        temp = 0
        if(movie in movie_id_to_ratings_train and movie in movie_id_to_ratings_test):
            for rating in movie_id_to_ratings_train[movie]:
                temp+=rating
            movie_id_to_average_rating_train[movie] = float(temp/len(movie_id_to_ratings_train[movie])) 

            for rating in movie_id_to_ratings_test[movie]:
                temp+=rating
            movie_id_to_average_rating_full[movie] = float(temp/(len(movie_id_to_ratings_train[movie])+len(movie_id_to_ratings_test[movie])))  

        elif(movie in movie_id_to_ratings_train):
            for rating in movie_id_to_ratings_train[movie]:
                temp+=rating
            movie_id_to_average_rating_train[movie] = float(temp/len(movie_id_to_ratings_train[movie]))
            movie_id_to_average_rating_full[movie] = movie_id_to_average_rating_train[movie]

        else:
            for rating in movie_id_to_ratings_test[movie]:
                temp+=rating
            movie_id_to_average_rating_full[movie] = float(temp/len(movie_id_to_ratings_test[movie]))
   

    full_user_to_ratings_full_transform = []
    full_user_to_ratings_full = []
    full_user_to_target_index_full = []


    full_user_to_movie_id_to_rating  = train_users.user_to_movie_id_to_rating + test_users.user_to_movie_id_to_rating
    full_user_to_target_movie_id = train_users.user_to_target_movie_id + test_users.user_to_target_movie_id


    #This is used to scale the ratings and store in train_users.user_to_ratings_full_transform and full_user_to_ratings_full_transform
    #This will transform the target movie ratings and unrated movies to zero

    #run once with only train data to train model
    #run again with train and test data to evaluate model...

    pre_svd(movie_id_to_average_rating_train, train_movies_in_order, train_users.user_to_ratings_full_transform, train_users.user_to_ratings_full, train_users.user_to_target_index_full, 
                train_users.user_to_movie_id_to_rating, train_users.user_to_target_movie_id)

    pre_svd(movie_id_to_average_rating_full, all_movies_in_order, full_user_to_ratings_full_transform, full_user_to_ratings_full, full_user_to_target_index_full, 
                full_user_to_movie_id_to_rating, full_user_to_target_movie_id)


    #In practice, there is a train and a test set, the train set is what the database has on record
    #the test data will usually be data that hasn't been seen before that can include any number of test users

    #When train_users.user_to_ratings_full_transform is used as the input of the svd function, 
    #svd_out_train is used to produce predictions used to train the model

    #When full_user_to_ratings_full_transform is used as the input of the svd function,
    #svd_out_full is used to produce predictions used to test the model
    

    #n = 20 proved to be close to the highest performing constant for the above configuration
    svd_out_train = svd_full(train_users.user_to_ratings_full_transform, 20, movie_id_to_average_rating_train)
    svd_out_full = svd_full(full_user_to_ratings_full_transform, 20, movie_id_to_average_rating_full)

    #here the smaller svd provides predictions used to train the mlp model
    for i in range(len(train_users.user_to_ratings_full_transform)):
        train_users.feature_3.append(svd_out_train[i][train_users.user_to_target_index_full[i]])

    #here the larger svd provides predictions used to test the mlp model
    for i in range(len(full_user_to_ratings_full_transform) - len(train_users.user_to_ratings_full_transform)):
        test_users.feature_3.append(svd_out_full[i+len(train_users.user_to_ratings_full_transform)][full_user_to_target_index_full[i]])

load_feature_3()

print(train_users.feature_3[0:5])
print(test_users.feature_3[0:5])



#used to see what the text data looks like...
# not applicable with dict to list change...
# file = open("test_dicts_1.txt", 'w', encoding="utf-8")
# file.write(json.dumps(user_to_movie_id_to_corpus_train))
# file.close()

# file = open("test_dicts_2.txt", 'w', encoding="utf-8")
# file.write(json.dumps(user_to_movie_id_to_corpus_test))
# file.close()


#this meight not be worth the deletion!!!
# del user_to_data_train
# del user_to_data_test


[3.4485433119510707, 3.594292957575485, 3.1103110332802335, 4.1514411264027355, 4.6693947158917695]
[3.315044275292965, 3.0242421457442292, 2.874113672682829, 4.563655346572882, 4.212991400499383]


In [13]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from ordered_set import OrderedSet
from collections import Counter



#what if populate 1 2 3 and 4 were combined???
#this would mean that user_to_movie_id_to_corpus and user_to_words_in_order would be changed to temporary instead
#to save memory...


# def populate_2(user_to_movie_id_to_corpus, user_to_words_in_order):
#     for i in range(len(user_to_movie_id_to_corpus)):
#         user_to_words_in_order.append(OrderedSet())
#         for movie_id in user_to_movie_id_to_corpus[i].keys():
#             for word in user_to_movie_id_to_corpus[i][movie_id]:
#                 user_to_words_in_order[i].add(word)

# populate_2(train_users.user_to_movie_id_to_corpus,  train_users.user_to_words_in_order)
# populate_2(test_users.user_to_movie_id_to_corpus,  test_users.user_to_words_in_order)



# def populate_3(user_to_movie_id_to_corpus, user_to_target_movie_id, user_to_movie_id_to_rating,
#     user_to_target_rating, user_to_ratings):
#     for i in range(len(user_to_movie_id_to_corpus)):
#         temp = []
#         cnt = 0
#         for movie_id in user_to_movie_id_to_corpus[i].keys():
#             if movie_id != user_to_target_movie_id[i]:
#                 temp.append(user_to_movie_id_to_rating[i][movie_id])
#             else:
#                 user_to_target_rating.append(user_to_movie_id_to_rating[i][movie_id])
#                 temp.append(-1)
#             cnt+=1
#         user_to_ratings.append(temp)


# populate_3(train_users.user_to_movie_id_to_corpus, train_users.user_to_target_movie_id, train_users.user_to_movie_id_to_rating,
#             train_users.user_to_target_rating, train_users.user_to_ratings)
# populate_3(test_users.user_to_movie_id_to_corpus, test_users.user_to_target_movie_id, test_users.user_to_movie_id_to_rating,
#             test_users.user_to_target_rating, test_users.user_to_ratings)



# def populate_4(user_to_word_counts_transformed, user_to_target_word_counts_transformed,
#                 user_to_movie_id_to_corpus, user_to_word_counts,
#                 user_to_target_movie_id, user_to_words_in_order, user_to_target_word_counts):
#     for i in range(len(user_to_movie_id_to_corpus)):
#         user_to_word_counts.append([])
#         user_to_word_counts_transformed.append([])
#         for movie_id in user_to_movie_id_to_corpus[i].keys():
#             if movie_id != user_to_target_movie_id[i]:
#                 temp_dict = Counter(user_to_movie_id_to_corpus[i][movie_id])
#                 temp_list = []
#                 avg = 0
#                 for word in user_to_words_in_order[i]:
#                     if word in temp_dict.keys():
#                         temp_list.append(temp_dict[word])
#                         avg+=temp_dict[word]
#                     else:
#                         temp_list.append(0)
#                 avg = float(avg/len(user_to_words_in_order[i]))
#                 user_to_word_counts[i].append(temp_list)
#                 user_to_word_counts_transformed[i].append([x - avg for x in temp_list])
#             else:

#                 temp_dict = Counter(user_to_movie_id_to_corpus[i][movie_id])
#                 temp_list = []
#                 avg = 0
#                 for word in user_to_words_in_order[i]:
#                     if word in temp_dict.keys():
#                         temp_list.append(temp_dict[word])
#                         avg+=temp_dict[word]
#                     else:
#                         temp_list.append(0)
#                 avg = float(avg/len(user_to_words_in_order[i]))
#                 user_to_target_word_counts.append(temp_list)
#                 user_to_target_word_counts_transformed.append([x - avg for x in temp_list])


# populate_4(train_users.user_to_word_counts_transformed, train_users.user_to_target_word_counts_transformed,
#            train_users.user_to_movie_id_to_corpus, train_users.user_to_word_counts, train_users.user_to_target_movie_id,
#        train_users.user_to_words_in_order, train_users.user_to_target_word_counts)

# populate_4(test_users.user_to_word_counts_transformed, test_users.user_to_target_word_counts_transformed,
#            test_users.user_to_movie_id_to_corpus, test_users.user_to_word_counts, 
#        test_users.user_to_target_movie_id, test_users.user_to_words_in_order, test_users.user_to_target_word_counts)

# del train_users.user_to_words_in_order
# del train_users.user_to_movie_id_to_corpus
# del test_users.user_to_words_in_order
# del test_users.user_to_movie_id_to_corpus



#now for each user, use the user_to_index_full and find the most similair users by omitting that index across the
#sim_matrix



#collaboritive filtering idea
#data structures:
#need a user to movies to ratings dictionary (done) (only needed for train data)
#need a ordered set of all movies (done) (only needed for train data)

#need to transform it into a user to list of all movies with user ratings or otherwise filled in ratings with sutiable average
#(the list needs to be in a consistent order across users)
#to fill in the averages we need a movie to average rating dictionary
#(note: if no other user has rated the movie then fill it in with the overall movie average)
#standardize the data row wise (why not columnwise???)
#filter the users that have rated the movie to predict
#then use cossine similairity on this set to find the most similair user the the chosen user
#ignore the movie rating to predict with the cossine similarity function

#Note: this is an expensive task and the number of users may have to be truncated before running



#ideas: 
#idea1: 
#collaborative filtering:
#https://towardsdatascience.com/predict-movie-ratings-with-user-based-collaborative-filtering-392304b988af
#https://www.geeksforgeeks.org/user-based-collaborative-filtering/#
#https://www.youtube.com/watch?v=3ecNC-So0r4&ab_channel=CodeHeroku
#idea2: 
#note: the model can be scored based on how close a predition is to a threshold of .5 

#idea 3:
#there may be a replacement for cossine similarity:
#perhaps it is not properly matching similair people and similair movies 
#maybe there is a replacement function

#idea 4: 
#there should be more train and test users

#idea 5: (basic model)
#https://www.kaggle.com/code/muhammadayman/recommendation-system-using-cosine-similarity#Data-Cleaning



#10 minutes for 2500 users

In [14]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from statistics import mean


# def predict(user, user_to_word_counts_transformed, user_to_target_word_counts_transformed,
#             user_to_target_word_counts, user_to_word_counts, user_to_ratings):


#     cosine_sim = cosine_similarity(X = user_to_word_counts_transformed[user] ,Y = [user_to_target_word_counts_transformed[user]])
    
#     cosine_sim = np.reshape(cosine_sim,  (len(cosine_sim)))

#     ratings = [user_to_ratings[user][x] for x in range(len(user_to_ratings[user])) if user_to_ratings[user][x] != -1]


#     #use the movie thats are most similair to the movie in question 

#     #option 1: 
#     # combined = zip(cosine_sim, ratings)
#     # combined = sorted(combined, key=lambda x: x[0], reverse=False)
#     # avg = 0
#     # nof = 10.0
#     # for i in range(int(nof)):
#     #     avg += combined[i][1]
#     # return float(avg/nof)

#     #option 2:
#     # avg = 0
#     # for i in range(len(ratings)):
#     #     avg += ratings[i]
#     # return float(avg/len(ratings))

#     #option 3: 
#     combined = zip(cosine_sim, ratings)
#     combined = sorted(combined, key=lambda x: x[0], reverse=True)
#     avg = 0
#     nof = 10.0
#     for i in range(int(nof)):
#         avg += combined[i][1]
#     return float(avg/nof)




# def populate_5(user_to_word_counts_transformed, user_to_target_word_counts_transformed,
#                user_to_movie_id_to_rating, feature_1,
#                 user_to_target_word_counts, user_to_word_counts,
#                 user_to_ratings):
#     for i in range(len(user_to_movie_id_to_rating)):
#         feature_1.append(predict(i, user_to_word_counts_transformed, user_to_target_word_counts_transformed,
#                                   user_to_target_word_counts, user_to_word_counts, user_to_ratings))



# populate_5(train_users.user_to_word_counts_transformed, train_users.user_to_target_word_counts_transformed,
#            train_users.user_to_movie_id_to_rating, train_users.feature_1,
#                 train_users.user_to_target_word_counts, train_users.user_to_word_counts,
#                 train_users.user_to_ratings)

# populate_5(test_users.user_to_word_counts_transformed, test_users.user_to_target_word_counts_transformed,
#            test_users.user_to_movie_id_to_rating, test_users.feature_1,
#                 test_users.user_to_target_word_counts, test_users.user_to_word_counts,
#                 test_users.user_to_ratings)

# del train_users.user_to_word_counts
# del test_users.user_to_word_counts
# del train_users.user_to_word_counts_transformed
# del test_users.user_to_word_counts_transformed
# del train_users.user_to_target_word_counts
# del test_users.user_to_target_word_counts
# del train_users.user_to_target_word_counts_transformed
# del test_users.user_to_target_word_counts_transformed
# del train_users.user_to_ratings
# del test_users.user_to_ratings


#note: for some reason feature 2 out performs feature 3 as a feature...

# for i in range(len(train_users.user_to_movie_id_to_rating)): 
#     if(len(movie_id_to_ratings_total[train_users.user_to_target_movie_id[i]])==1):
#         train_users.feature_2.append(overall_average_train)
#     else:
#         train_users.feature_2.append(float(((movie_id_to_average_rating_train[train_users.user_to_target_movie_id[i]]
#                         *len(movie_id_to_ratings_total[train_users.user_to_target_movie_id[i]]))
#                         -train_users.user_to_movie_id_to_rating[i][train_users.user_to_target_movie_id[i]])
#                         /(len(movie_id_to_ratings_total[train_users.user_to_target_movie_id[i]])-1)))


# for i in range(len(test_users.user_to_movie_id_to_rating)):
#     if(test_users.user_to_target_movie_id[i] not in movie_id_to_ratings_total.keys()):
#         test_users.feature_2.append(overall_average_train)
#     else:
#         test_users.feature_2.append(movie_id_to_average_rating_train[test_users.user_to_target_movie_id[i]])



# def populate_6(user_to_ratings_full_transform, user_to_ratings_full, user_to_target_index_full, 
#                user_to_movie_id_to_rating, user_to_target_movie_id):
#     for i in range(len(user_to_movie_id_to_rating)):
#         ratings = []
#         index = 0
#         for movie_id in movies_in_order:
#             if movie_id == user_to_target_movie_id[i]:
#                 user_to_target_index_full.append(index)
#             if movie_id in user_to_movie_id_to_rating[i].keys():
#                 ratings.append(user_to_movie_id_to_rating[i][movie_id])
#             elif movie_id in movie_id_to_average_rating_train.keys():
#                 ratings.append(movie_id_to_average_rating_train[movie_id])
#             else:   
#                 ratings.append(overall_average_train)
#             index +=1
#         user_to_ratings_full.append(ratings)
#         user_to_ratings_full_transform.append([x - user_to_average_rating[i] for x in ratings])


# populate_6(train_users.user_to_ratings_full_transform, train_users.user_to_ratings_full, train_users.user_to_target_index_full, train_users.user_to_movie_id_to_rating, train_users.user_to_target_movie_id)
# populate_6(test_users.user_to_ratings_full_transform, test_users.user_to_ratings_full, test_users.user_to_target_index_full, test_users.user_to_movie_id_to_rating, test_users.user_to_target_movie_id)


# del train_users.user_to_movie_id_to_rating
# del test_users.user_to_movie_id_to_rating
# del train_users.user_to_target_movie_id
# del test_users.user_to_target_movie_id

#why is feature 3 outperformed by feature 2 ???

# def populate_7(user_to_ratings_full_transform, user_to_ratings_full, user_to_target_index_full, feature_3):
#     for i in range(len(user_to_ratings_full)):
#         list_of_list_of_ratings = []
#         sample_ratings = []
#         for j in range(len(train_users.user_to_ratings_full)):
#             if i != j:
#                 sample_ratings.append(train_users.user_to_ratings_full[j][user_to_target_index_full[i]])
#                 ratings = [train_users.user_to_ratings_full_transform[j][x] 
#                           for x 
#                           in range(len(train_users.user_to_ratings_full_transform[j])) 
#                           if x != user_to_target_index_full[i]]
#                 list_of_list_of_ratings.append(ratings)

#         user_ratings = [user_to_ratings_full_transform[i][x] 
#                     for x 
#                     in range(len(user_to_ratings_full_transform[i])) 
#                     if x != user_to_target_index_full[i]]


#         sim  = cosine_similarity(X = list_of_list_of_ratings, Y = [user_ratings])
#         sim = np.reshape(sim,  (len(sim)))

#         combined = zip(sim, sample_ratings)
#         combined = sorted(combined, key=lambda x: x[0], reverse = True)

#         avg = 0
#         nof = 10.0
#         for k in range(int(nof)):
#             avg+= combined[k][1]

#         feature_3.append(float(avg/nof))
    

# populate_7(train_users.user_to_ratings_full_transform, train_users.user_to_ratings_full, train_users.user_to_target_index_full, train_users.feature_3)
# populate_7(test_users.user_to_ratings_full_transform, test_users.user_to_ratings_full, test_users.user_to_target_index_full, test_users.feature_3)

# del train_users.user_to_ratings_full_transform
# del test_users.user_to_ratings_full_transform
# del train_users.user_to_ratings_full
# del test_users.user_to_ratings_full
# del train_users.user_to_target_index_full
# del test_users.user_to_target_index_full


#adjusted cossine simlairity
#https://stackoverflow.com/questions/40716459/choice-between-an-adjusted-cosine-similarity-vs-regular-cosine-similarity
#https://github.com/csaluja/JupyterNotebooks-Medium/blob/master/CF%20Recommendation%20System-Examples.ipynb?source=post_page-----ecbffe1c20b1--------------------------------
#https://towardsdatascience.com/collaborative-filtering-based-recommendation-systems-exemplified-ecbffe1c20b1


#note: in the item-based collabortive filtering the means is implementaed item-wise
#this means with user-based collaboritve filtering the mean is implementad user-wise


#idea 1:
#use diferrent data structures...
#numpy arrays...
#https://www.geeksforgeeks.org/python-lists-vs-numpy-arrays/#
#https://stackoverflow.com/questions/29839350/numpy-append-vs-python-append
#https://www.geeksforgeeks.org/python-convert-list-to-python-array/


#idea 2:
#make use of both feature_2 and feature_3

#idea 3: would it be possible to convert many of the main dictionaries to lists or numpy arrays
#note: numpy arrays cant have variable number of items for a dimension
#but this can be over come by filling missing values
#this mean many matrices would be sparse
#there is also alot of appending to lists which is better for lists


#idea 4: would it be helpful to design my own data structures???

#idea 5: would it be helpful to have features like the average 10 closest in similairy and the 10 farthest in similarity???

#idea 6: before trying this there could be better data strcuture to use that would speed up the process



In [23]:
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


#average the performance results for a number of models with identical inputs
def test_parameters(nof_runs, layers, train_input_features, test_input_features):
    train_inputs = [list(pair) for pair in train_input_features]
    test_inputs = [list(pair) for pair in test_input_features]
    return average_results(nof_runs, layers, train_inputs, test_inputs)
    

def average_results(nof_runs, layers, train_inputs, text_inputs):
    no_rounding = 0
    rounding = 0
    for _ in range(nof_runs):
        #best performance analysis is analysis_1
        pair = analysis_1(layers, train_inputs, text_inputs)
        no_rounding+=pair[0]
        rounding+=pair[1]
    return float(no_rounding/nof_runs), float(rounding/nof_runs)


#no scaling (best performance):
def analysis_1(layers, train_inputs, test_inputs):
    #build and train model
    reg = MLPRegressor(hidden_layer_sizes = layers, solver = "adam",  max_iter = 1000)
    reg.fit(train_inputs, train_users.user_to_target_rating)

    #show importance of different inputs features to the model
    results = permutation_importance(reg, train_inputs, train_users.user_to_target_rating)
    print(results["importances_mean"])

    #make predictions
    predictions = reg.predict(test_inputs)

    #test with and without roundings...
    #in a sense this is logical sense becasue the actual ratings a user makes must be divisable by .5 
    rounded_predictions = []
    for item in predictions:
        rounded_predictions.append(float(round(item*2)/2.0))

    #evaluation metric 1:
    return(r2_score(test_users.user_to_target_rating, predictions), 
        r2_score(test_users.user_to_target_rating, rounded_predictions))

    #evaluation metric 2:
    # return(mean_squared_error(test_users.user_to_target_rating, predictions), 
    #         mean_squared_error(test_users.user_to_target_rating, rounded_predictions))

#scale inputs and targets:
def analysis_2(layers, train_inputs, test_inputs):
    #scale input features
    train_inputs_scaled = StandardScaler().fit_transform(train_inputs)

    #scale target values
    target_scalar = StandardScaler()
    true_rating_train_scaled = target_scalar.fit_transform(np.reshape(train_users.user_to_target_rating, (-1, 1)))
    true_rating_train_scaled = np.reshape(true_rating_train_scaled, len(true_rating_train_scaled))

    #build and train model
    reg = MLPRegressor(hidden_layer_sizes = layers, solver = "adam",  max_iter = 1000)
    reg.fit(train_inputs_scaled, true_rating_train_scaled)

    #show importance of different inputs features...
    results = permutation_importance(reg, train_inputs_scaled,true_rating_train_scaled)
    print(results["importances_mean"])

    #scale inputs features
    test_inputs_scaled = StandardScaler().fit_transform(test_inputs)

    #predict the scaled verison of ouptuts
    scaled_predictions = reg.predict(test_inputs_scaled)

    #get actual predictions from scaled predictions...
    predictions = target_scalar.inverse_transform(scaled_predictions.reshape(-1, 1))
    predictions = list(predictions.reshape(len(predictions)))

    #test with and without roundings...
    #in a sense this is logical sense becasue the actual ratings a user makes must be divisable by .5 
    rounded_predictions = []
    for item in predictions:
        rounded_predictions.append(float(round(item*2)/2.0))

    #evaluation metric 1:
    return(r2_score(test_users.user_to_target_rating, predictions), 
        r2_score(test_users.user_to_target_rating, rounded_predictions))

    #evaluation metric 2:
    # return(mean_squared_error(test_users.user_to_target_rating, predictions), 
    #         mean_squared_error(test_users.user_to_target_rating, rounded_predictions))

#only scale inputs:
def analysis_3(layers, train_inputs, test_inputs):
    #scale input features
    train_inputs_scaled = StandardScaler().fit_transform(train_inputs)

    #build and train model
    reg = MLPRegressor(hidden_layer_sizes = layers, solver = "adam",  max_iter = 1000)
    reg.fit(train_inputs_scaled, train_users.user_to_target_rating)

    #show importance of different inputs features...
    results = permutation_importance(reg, train_inputs_scaled, train_users.user_to_target_rating)
    print(results["importances_mean"])

    #scale inputs features
    test_inputs_scaled = StandardScaler().fit_transform(test_inputs)

    #predict the scaled verison of ouptuts
    predictions = reg.predict(test_inputs_scaled)

    #test with and without roundings...
    #in a sense this is logical sense becasue the actual ratings a user makes must be divisable by .5 
    rounded_predictions = []
    for item in predictions:
        rounded_predictions.append(float(round(item*2)/2.0))

    #evaluation metric 1:
    return(r2_score(test_users.user_to_target_rating, predictions), 
        r2_score(test_users.user_to_target_rating, rounded_predictions))

    #evaluation metric 2:
    # return(mean_squared_error(test_users.user_to_target_rating, predictions), 
    #         mean_squared_error(test_users.user_to_target_rating, rounded_predictions))



#the current test is the average of the r2 scores for 100 different models trained on the same input
#the hidden layers are (10,10,10) and the best combinatio of inputs features(feature_2 and feature_3) are used
print(test_parameters(100, (5,10,10), 
    zip(train_users.feature_1, train_users.feature_3),
      zip(test_users.feature_1, test_users.feature_3)))


#this shows the side by side comparision between all the features and the actual rating
#each feature provides a reasonable guess of the target rating
#the combination of the feature used above (feature_2 and feature_3) proves stronger than any feature alone and any other combination of features
# print(test_users.feature_1)
# print(test_users.feature_2)
print(test_users.feature_3)
# print(test_users.user_to_target_rating)

#n = 15
#feature_2 and feature_3:
#r_2 scores:
#(0.3551835602886015, 0.33621944330534004)

#feature_1 and feature_3:
#r_2 scores:
#(0.3539859098451409, 0.3389716329978715)

#feature_1:
#r_2 scores:
#(0.13997553160939374, 0.12017831016064674)

#feature_2
#r_2 scores:
#(0.1464331784040238, 0.12889933523935157)

#feature_3:
#r_2 scores:
#(0.31424317374455685, 0.2999151056317528)


# n = 20
# feature_2 and feature_3:
# (0.3652700035771331, 0.3439539791916036)

# feature_1 and feature_3:
# (0.36616132482898556, 0.3435471003807131)
# (0.36607210775528126, 0.34202322407954866)


#n = 25
# feature_1 and feature_3:
#(0.35985921904576285, 0.3294790735702145)

#n = 30
# feature_1 and feature_3:
# (0.35995387004644597, 0.340802587646878)

#(0.3657076246215354, 0.3445834898046792)

#ideas:
#what if stopwords were included or the words have more or less filtering???
#for content based reccomendation, may need to focus on words from certain sections to not overwelm the model with useless information!!!
#k-means clusting can be effected by the random initialization...
#this is a reason that there is substantial variation in perfromance
#note: there are some predictions over 5 stars???
#idea: what about only focusing on movies with a specific number of ratings???
#what if there are more ratings per user???
#what if boolena values were used for content baed items instead of counts???



#what if there was a way to not limit content based filtering to movies a single user watched???

#even even even more (done):
#https://www.kaggle.com/code/cast42/simple-svd-movie-recommender

#even even more svd:
#https://analyticsindiamag.com/singular-value-decomposition-svd-application-recommender-system/

#even more svd:
#https://towardsdatascience.com/beginners-guide-to-creating-an-svd-recommender-system-1fd7326d1f65

#more svd:
#https://machinelearningmastery.com/using-singular-value-decomposition-to-build-a-recommender-system/

#even even more content base recomendations:
#https://www.kaggle.com/code/ibtesama/getting-started-with-a-movie-recommendation-system

#even more content base recomendations:
#https://towardsdatascience.com/the-4-recommendation-engines-that-can-predict-your-movie-tastes-109dc4e10c52

#more content based reccomendation:
#https://medium.com/web-mining-is688-spring-2021/content-based-movie-recommendation-system-72f122641eab#:~:text=Content%20Based%20Recommendation%20System%3A%20It,a%20show%20similar%20to%20it.

#content based reccomendation:
#https://medium.com/geekculture/creating-content-based-movie-recommender-with-python-7f7d1b739c63

#methods used to guess ratings
#https://web.eecs.umich.edu/~cscott/past_courses/eecs545f11/projects/AsendorfMcgaffinPressSchwartz.pdf

#svd...
#https://www.youtube.com/watch?v=8wLKuscyO9I&ab_channel=SundogEducationwithFrankKane

#pca...
#https://www.youtube.com/watch?v=fkf4IBRSeEc&ab_channel=SteveBrunton

#eigenvalues...
#https://www.youtube.com/watch?v=OA6CkChbe0Q&ab_channel=AndrewMisseldine

#more pca:
#https://www.youtube.com/watch?v=TJdH6rPA-TI&ab_channel=Computerphile

#svd lecture:
#https://www.youtube.com/watch?v=rFemvJgXY7E&ab_channel=Tech4Trends

#svd with tf-idf
#https://www.kaggle.com/code/parnianmalekian/svd-and-its-application-in-tf-idf


[0.11446095 0.49999414]
[0.11272463 0.5040759 ]
[0.11801377 0.52338134]
[0.09712008 0.53209132]
[0.11574293 0.50845833]
[0.08118392 0.58226655]
[0.09825649 0.52029808]
[0.11339681 0.53787719]
[0.10578583 0.50357237]
[0.09618796 0.51974685]
[0.10250218 0.54391883]
[0.09336296 0.5390289 ]
[0.10382674 0.52104898]
[0.10197849 0.5336034 ]
[0.10023585 0.52182004]
[0.11313583 0.54078518]
[0.09743121 0.53025303]
[0.1085507  0.51934762]
[0.10248263 0.52958871]
[0.10832568 0.55445313]
[0.10057431 0.53043693]
[0.08832491 0.57258852]
[0.10748734 0.48725584]
[0.06720502 0.59019864]
[0.10149316 0.53024563]
[0.10502173 0.52800514]
[0.13532738 0.48313905]
[0.09408332 0.52949571]
[0.11032573 0.50158475]
[0.10103751 0.50389366]
[0.09654603 0.53543992]
[0.10461434 0.53457329]
[0.11241167 0.54713019]
[0.10800685 0.53139944]
[0.09663294 0.52401869]
[0.09203704 0.55486694]
[0.10420262 0.52416764]
[0.09502345 0.56178625]
[0.11417397 0.51591047]
[0.10467282 0.51806216]
[0.102245   0.53279004]
[0.11476225 0.51

In [16]:
#this cell is for notes and models with scaling and no scaling...


#analysis without feature scaling...
#feature scaling:
#https://analyticsindiamag.com/why-data-scaling-is-important-in-machine-learning-how-to-effectively-do-it/#:~:text=Scaling%20the%20target%20value%20is,learn%20and%20understand%20the%20problem.&text=Scaling%20of%20the%20data%20comes,algorithms%20in%20the%20data%20set.
#https://towardsdatascience.com/collaborative-filtering-based-recommendation-systems-exemplified-ecbffe1c20b1


# with scaling start:

# def test_parameters(layers, train_input_features, test_input_features):
#     train_inputs = [list(pair) for pair in train_input_features]
#     test_inputs = [list(pair) for pair in test_input_features]

#     return average_results(20, layers, train_inputs, test_inputs)
    

# def average_results(nof_runs, layers, train_inputs, text_inputs):
#     no_rounding = 0
#     rounding = 0
#     for _ in range(nof_runs):
#         #issue here
#         pair = analysis(layers, train_inputs, text_inputs)
#         no_rounding+=pair[0]
#         rounding+=pair[1]
#     return float(no_rounding/nof_runs), float(rounding/nof_runs)


# def analysis(layers, train_inputs, test_inputs):
#     #scale input features
#     #scalar 1:
#     input_scalar = StandardScaler()
#     train_inputs_scaled = input_scalar.fit_transform(train_inputs)

#     #scale target values
#     #scalar 2:
#     target_scalar = StandardScaler()
#     true_rating_train_scaled = target_scalar.fit_transform(np.array(train_users.user_to_target_rating).reshape(-1, 1))
#     true_rating_train_scaled = np.reshape(true_rating_train_scaled, len(true_rating_train_scaled))

#     #build and train model
#     reg = MLPRegressor(hidden_layer_sizes = layers, solver = "adam",  max_iter = 1000)
#     reg.fit(train_inputs_scaled, true_rating_train_scaled)

#     #show importance of different inputs features...
#     results = permutation_importance(reg, train_inputs_scaled,true_rating_train_scaled)
#     print(results["importances_mean"])

#     #scale inputs features
#     #scalar 3: 
#     input_scalar = StandardScaler()
#     test_inputs_scaled = input_scalar.fit_transform(test_inputs)

#     #predict the scaled verison of ouptuts
#     scaled_predictions = reg.predict(test_inputs_scaled)

#     #change back into regular outputs to be tested with target scalar above...
#     #not sure if this is correct???
#     #would it be better to use a new scalar object???
#     #need to know the mean and the std dev of the orginal predictions
#     #scalar 4:
#     predictions = target_scalar.inverse_transform(np.array(scaled_predictions).reshape(-1, 1))

#     #test with and without roundings...
#     rounded_predictions = []
#     for item in predictions:
#         rounded_predictions.append(float(round(item[0]*2)/2.0))

#     #evaluation metric 1
#     return(r2_score(test_users.user_to_target_rating, predictions), 
#         r2_score(test_users.user_to_target_rating, rounded_predictions))

#     #evaluation metric 2
#     # return(mean_squared_error(test_users.user_to_target_rating, predictions), 
#     #         mean_squared_error(test_users.user_to_target_rating, rounded_predictions))


# print(test_parameters((10,10,10), 
#       zip(train_users.feature_1, train_users.feature_3),
#       zip(test_users.feature_1, test_users.feature_3)))


# print(test_users.feature_1)
# print(test_users.feature_3)
# print(test_users.user_to_target_rating)

# ...with scaling done


#without scaling:

# def test_parameters(layers, train_input_features, test_input_features):
#     train_inputs = [list(pair) for pair in train_input_features]
#     test_inputs = [list(pair) for pair in test_input_features]

#     return average_results(20, layers, train_inputs, test_inputs)
    


# def average_results(nof_runs, layers, train_inputs, text_inputs):
#     no_rounding = 0
#     rounding = 0
#     for _ in range(nof_runs):
#         #issue here
#         pair = analysis(layers, train_inputs, text_inputs)
#         no_rounding+=pair[0]
#         rounding+=pair[1]
#     return float(no_rounding/nof_runs), float(rounding/nof_runs)


# def analysis(layers, train_inputs, test_inputs):
#     #build and train model
#     reg = MLPRegressor(hidden_layer_sizes = layers, solver = "adam",  max_iter = 1000)
#     reg.fit(train_inputs, train_users.user_to_target_rating)

#     #show importance of different inputs features...
#     results = permutation_importance(reg, train_inputs, train_users.user_to_target_rating)
#     print(results["importances_mean"])


#     predictions = reg.predict(test_inputs)

#     #testing if needed...
#     # predictions = np.array(predictions).reshape(-1, 1)

#     #test with and without roundings...
#     rounded_predictions = []
#     for item in predictions:
#         # rounded_predictions.append(float(round(item[0]*2)/2.0))
#         rounded_predictions.append(float(round(item*2)/2.0))

#     #evaluation metric 1:
#     return(r2_score(test_users.user_to_target_rating, predictions), 
#         r2_score(test_users.user_to_target_rating, rounded_predictions))

#     #evaluation metric 2:
#     # return(mean_squared_error(test_users.user_to_target_rating, predictions), 
#     #         mean_squared_error(test_users.user_to_target_rating, rounded_predictions))


# print(test_parameters((10,10,10), 
#       zip(train_users.feature_1, train_users.feature_3),
#       zip(test_users.feature_1, test_users.feature_3)))

# print(test_users.feature_1)
# print(test_users.feature_3)
# print(test_users.user_to_target_rating)

#done with scaling...


#now for the user comparison option (need user to list of movie ratings)
#fill in ratings that the user hasn't watched with the method above
#then cluster the users by their ratings

#note: agglomerative clustering might make more sense here since k-means has random init for centroids...
#note: to guess a new users rating requires that none of that users ratings have been used to train the model
#The data needs to be split into test and train before modeling the algorithm on the train data

#Training process:
#split data into test and train data
#proceed with train data...
#cluster movies by the tokens with range for k
#cluster users by the ratings with range for k and (fill in ratings for movies a users hasn't watched with some guess)
#guess: this can be obtained by clustering the movies that the user has watched...
#for each movie the user hasn't watched find the cluster that it belongs to with the highest possible k value
#that the user has at least one movie belonging to one of the clusters and then take the average of those movies
#this is exactly like a later training step excpet it is applied to all the movies the user watched

#for a single randomly chosen movie from each user in the trainging data...

#find the cluster the movie belongs to 
#find the movies part of that same cluster that the user has scored at the highest possible k value
#take the average score of these movies
#find the cluster the user belongs to
#find the average rating of the movie for users in that cluster at the highest possible k value
#train an mlp model with both averages and perhaps some extra statistics as features...
#using the given movie ratings as actuals


#The process of predicting a rating:
#1. find the cluster the movie belongs to 
#2. find the movies part of that same cluster that the user has scored at the highest possible k value
#3. take the average score of these movies
#4. find the cluster the user belongs to
#5. find the average rating of the movie for users in that cluster at the highest possible k value
#6. input into the trained mlp model both averages and perhaps some extra statistics
#7. make predictions and test against the randomly chosen movies actual ratings


#summary:
#find cluster for movie -> find movies part of the same clusters that the users rated -> average
#question: are the clusters unique to the movies the user has watched or to all movies???
#what is the technical difference???
#is this the same as finding the most simimlair movie the user rated and copying the rating???

#find cluster for user -> find the ratings for the movie by people in the same cluster -> average

#other avenues considered:
#idea 1:
#for the first process, instead of averaging the movies that only the user rated, find other users that are...
#like the user in question and find the average for that movie cluster
#Problem: it is better to get the users raw opionion rather than generalizing it to some like minded users
#there is an extra costly step to this
#idea 2: 
#for the second process, instead of finding the average rating for the movie in the same cluster of users...
#also find the average rating of movies that are like the movie in question 
#Problem, it is better to get the movies rating itself as it would be the most accurate indicator
#there is an extra costly step to this




#test with overall averges:
#note: users have between 50 and 75 ratings each
#note: there are 250 users who are taken into account 
#note: seed int is one for above cells (cells part of creating the csv file)

#with seed int == 3, 4 taking overall averages: 0.09565753948597455
#with seed_int == 1, 2 taking overall averages: 0.070404868516315
#with seed_int == 2, 5 taking overall averages: 0.11310085954932936
#with seed_int == 4, 6 taking overall averages: 0.07125374341347135
#with seed_int == 5, 4 taking overall averages: 0.17736444913943628
#compute time: 11 minutes


#test with users related movies:
#is there a magic proportion of movies to average???
#note: this is taking around the same time as the above tests meaning 
#there could be more users to include in analysis with little increase in runtime
#k fold cross validation could be effective
#https://www.youtube.com/watch?v=TIgfjmp-4BA&ab_channel=Udacity

#effect of choosing a random seed...
#https://towardsdatascience.com/how-to-use-random-seeds-effectively-54a4cd855a79

#try tinkering with the number of similair movies to average
